In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt
import os

2021-08-01 12:34:23.253198: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-01 12:34:23.253235: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
BASE_DIR = os.getcwd()
ratings_file = BASE_DIR+'/Data/filtered_user.csv'
df = pd.read_csv(ratings_file)

In [3]:
user_ids = df["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
artists = df["artist_id"].unique().tolist()
artist2artist_encoded = {x: i for i, x in enumerate(artists)}
artist_encoded2artist = {i: x for i, x in enumerate(artists)}
df["user"] = df["user_id"].map(user2user_encoded)
df["artists"] = df["artist_id"].map(artist2artist_encoded)

num_users = len(user2user_encoded)
num_artists = len(artist_encoded2artist)
df["rating"] = df["rating"].values.astype(np.float32)
# min and max ratings will be used to normalize the ratings later
min_rating = min(df["rating"])
max_rating = max(df["rating"])

print(
    "Number of users: {}, Number of Artists: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_artists, min_rating, max_rating
    )
)

Number of users: 19017, Number of Artists: 11488, Min rating: 0.0, Max rating: 255.0


In [4]:
df.head(100)

,user_id,artist_id,rating,name,user,artists
0,1,1000125,90.0,2pac,0,0
1,1,1006373,100.0,cypress hill,0,1
2,1,1006978,90.0,deftones,0,2
3,1,1007035,100.0,delinquent habits,0,3
4,1,1007098,100.0,depeche mode,0,4
...,...,...,...,...,...,...
95,5,1013384,0.0,jimmy eat world,4,87
96,5,1013412,100.0,jodeci,4,88
97,5,1013565,100.0,jon b,4,89
98,5,1014120,100.0,r kelly,4,90


In [5]:
df = df.sample(frac=1, random_state=42)
x = df[["user", "artist_id"]].values
# Normalize the targets between 0 and 1. Makes it easy to train.
y = df["rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
# Assuming training on 90% of the data and validating on 10%.
train_indices = int(0.9 * df.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

In [6]:
len(x) #1013012

1068406

In [7]:
EMBEDDING_SIZE = 50


class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_artistss, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_artists = num_artists
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            len(x)+1,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(len(x), 1)
        self.artist_embedding = layers.Embedding(
            len(x)+1,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.artist_bias = layers.Embedding(len(x), 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        artist_vector = self.artist_embedding(inputs[:, 1])
        artist_bias = self.artist_bias(inputs[:, 1])
        dot_user_artist = tf.tensordot(user_vector, artist_vector, 2)
        # Add all the components (including bias)
        x = dot_user_artist + user_bias + artist_bias
        # The sigmoid activation forces the rating to between 0 and 1
        return tf.nn.sigmoid(x)


model = RecommenderNet(len(x), len(x), EMBEDDING_SIZE)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(learning_rate=0.001)
)


2021-08-01 12:34:29.091485: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-01 12:34:29.091556: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-01 12:34:29.091600: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (shahanex-HP-ENVY-x360-Convertible-15m-cp0xxx): /proc/driver/nvidia/version does not exist
2021-08-01 12:34:29.092160: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
len(x_val) # x_train 1024138 961565  x_val 11488 106841

106841

In [9]:
num_users   #indices[0] = 1.017.354 is not in [0, 11488)

19017

In [10]:
num_artists

11488

In [11]:
EMBEDDING_SIZE

50

In [12]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=5,
    verbose=1,
    validation_data=(x_val, y_val),
)


2021-08-01 12:34:32.495064: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-01 12:34:32.514443: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2195890000 Hz


Epoch 1/5


InvalidArgumentError:  indices[5] = 1098595 is not in [0, 1068406)
	 [[node recommender_net/embedding_3/embedding_lookup (defined at tmp/ipykernel_5714/1267955913.py:29) ]] [Op:__inference_train_function_1021]

Errors may have originated from an input operation.
Input Source operations connected to node recommender_net/embedding_3/embedding_lookup:
 recommender_net/embedding_3/embedding_lookup/637 (defined at home/shahanex/anaconda3/lib/python3.8/contextlib.py:113)

Function call stack:
train_function


### plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()


In [ ]:
movie_df = pd.read_csv(BASE_DIR / "filtered_artist.csv")

# Let us get a user and see the top recommendations.
user_id = df.userId.sample(1).iloc[0]
movies_watched_by_user = df[df.userId == user_id]
movies_not_watched = movie_df[
    ~movie_df["movieId"].isin(movies_watched_by_user.movieId.values)
]["movieId"]
movies_not_watched = list(
    set(movies_not_watched).intersection(set(movie2movie_encoded.keys()))
)
movies_not_watched = [[movie2movie_encoded.get(x)] for x in movies_not_watched]
user_encoder = user2user_encoded.get(user_id)
user_movie_array = np.hstack(
    ([[user_encoder]] * len(movies_not_watched), movies_not_watched)
)
ratings = model.predict(user_movie_array).flatten()
top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_movie_ids = [
    movie_encoded2movie.get(movies_not_watched[x][0]) for x in top_ratings_indices
]

print("Showing recommendations for user: {}".format(user_id))
print("====" * 9)
print("Movies with high ratings from user")
print("----" * 8)
top_movies_user = (
    movies_watched_by_user.sort_values(by="rating", ascending=False)
    .head(5)
    .movieId.values
)
movie_df_rows = movie_df[movie_df["movieId"].isin(top_movies_user)]
for row in movie_df_rows.itertuples():
    print(row.title, ":", row.genres)

print("----" * 8)
print("Top 10 movie recommendations")
print("----" * 8)
recommended_movies = movie_df[movie_df["movieId"].isin(recommended_movie_ids)]
for row in recommended_movies.itertuples():
    print(row.title, ":", row.genres)
